In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re,string

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/storydata/rsp_story_data.csv


In [4]:
df = pd.read_csv("/kaggle/input/storydata/rsp_story_data.csv")
df = df.drop(columns=['ExternalId','ReleaseUId','Unnamed: 0'])

In [5]:
df

,Title,Description,AcceptanceCriteria
0,As a RSP[UI] user I must be able to view RSP W...,"As a RSP User, I should be able to view the ...",NaN
1,As RSP[UI] user when I click on the bubble in ...,Clicking the bubble in the landing page cha...,The bubble chart is clickable Clicking the bub...
2,As a R360 phoenix user Audit Fields present in...,"As a part of GDPR compliance, it is required t...","The audit fields CreatedBy, CreatedAtSourceBy..."
3,Technical Story - To update Modified On while ...,As part of Phoenix Feature Implementation 1718...,NaN
4,As RSP[UI] user I should get the summary of de...,The widget should have one line displaying ...,Widget should show the right value of Objects\...
...,...,...,...
12428,Data inactivation for past releases in splashb...,The configured Attention area and scrum talkin...,The data on the backend should be cleared for ...
12429,As a phoenix user Audit Fields present in Phoe...,"As a part of GDPR compliance, it is required t...","The existing data to be encrypted . also, the ..."
12430,As a phoenix user Audit Fields present in Phoe...,"As a part of GDPR compliance, it is required t...","The existing data to be encrypted . also, the ..."
12431,VA | Confidential popup vulnerability fix,While creating cookie need to add below highli...,Cookie creation details to be modified to incl...


In [6]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [7]:
# Load the tokenizer and the GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [8]:
df['StoryTitleDescription'] = df['Title'] + " " + df['Description']
df = df.drop(columns=['Title','Description'],axis=1)

In [9]:
df['Acc'] = df['AcceptanceCriteria']
df = df.drop(columns=['AcceptanceCriteria'])

In [10]:
df = df.fillna(" ")

In [11]:
def remove_punctuation(input):
    #puctuation_pattern = re.compile(r'[%s]'%string.punctuation
    input=re.sub(r'[^\w\s.]', '', input)
    #puctuation_pattern=re.compile(r'[!"#$%&\'()*+-/:;<=>?@[\]^_`{|}~]', re.UNICODE)
    input = re.sub(r'[^\w\s.]', '', input)
    input = input.replace('  ',' ')
    input = input.replace("\n",' ')
    return input

In [12]:
df['StoryTitleDescription'] = df['StoryTitleDescription'].apply(lambda x : remove_punctuation(x))
df['Acc'] = df['Acc'].apply(lambda x : remove_punctuation(x))

In [13]:
df.head()

,StoryTitleDescription,Acc
0,As a RSPUI user I must be able to view RSP Wid...,
1,As RSPUI user when I click on the bubble in Ho...,The bubble chart is clickable Clicking the bub...
2,As a R360 phoenix user Audit Fields present in...,The audit fields CreatedBy CreatedAtSourceByU...
3,Technical Story To update Modified On while up...,
4,As RSPUI user I should get the summary of defe...,Widget should show the right value of Objects ...


In [14]:
with open("/kaggle/working/rsp_story_new.txt", 'w', encoding = 'utf-8') as f:
    for index,row in df.iterrows():
        text = row['StoryTitleDescription'] + "| " + row['Acc']
        f.write(text + '\n')

In [15]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [16]:
# Load pre-trained GPT-Neo model and tokenizer
#model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-1.3B')
#tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-1.3B')
# Load the tokenizer and the GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Prepare the training data
user_stories = ["As a customer, I want to be able to create an account.", "As a customer, I want to be able to search for products."]
acceptance_criteria = ["The account creation page should have fields for username, password, and email address.", "The search results should be sorted by relevance."]
inputs = []
outputs = []

for i in range(len(user_stories)):
    input_text = "User story: " + user_stories[i] + "\n"
    output_text = "Acceptance criteria: " + acceptance_criteria[i] + "\n"
    inputs.append(input_text)
    outputs.append(output_text)




In [ ]:
# Tokenize the data
input_tokens = tokenizer.batch_encode_plus(inputs, pad_to_max_length=True, return_tensors="pt")
output_tokens = tokenizer.batch_encode_plus(outputs, pad_to_max_length=True, return_tensors="pt")

# Format the data as input-output pairs
input_ids = input_tokens["input_ids"]
output_ids = output_tokens["input_ids"]
attention_mask = input_tokens["attention_mask"]

# Train the model on the input-output pairs
model.train()
for epoch in range(num_epochs):
    loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=output_ids).loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


In [17]:
# Load your dataset of user stories and acceptance criteria
dataset = TextDataset(tokenizer=tokenizer, file_path='/kaggle/working/rsp_story_new.txt', block_size=128)


/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:57: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [18]:
# Set up training arguments and data collator
training_args = TrainingArguments(
    output_dir='/kaggle/working/results',
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    save_steps=5000,
    disable_tqdm =False,
    save_total_limit=2
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [19]:
from sklearn.model_selection import train_test_split
train_val, test = train_test_split(dataset, test_size=0.2, random_state=42)
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

In [ ]:
# Create trainer object and fine-tune the model on your dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    data_collator=data_collator
)
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,3.275700
1000,2.445800
1500,2.126700
2000,1.827700
2500,1.766100
3000,1.607300
3500,1.464400
4000,1.356500
4500,1.289400
5000,1.242700
